In [125]:
import pandas as pd
import re
import numpy as np

In [126]:
child = pd.read_csv("./data/전국어린이보호구역표준데이터.csv", encoding = "euc-kr")

In [127]:
idx_lst = []
for idx in range(len(child)):
    if "서울" in child["제공기관명"][idx]:
        idx_lst.append(idx)
child = child.loc[idx_lst, :].reset_index(drop = True)
child["옐로카펫_설치일자"] = 0
child["보호구역_지정년도"] = 0

In [128]:
child.head(2)

,시설종류,대상시설명,소재지도로명주소,소재지지번주소,위도,경도,관리기관명,관할경찰서명,CCTV설치여부,CCTV설치대수,보호구역도로폭,데이터기준일자,제공기관코드,제공기관명,옐로카펫_설치일자,보호구역_지정년도
0,유치원,사랑,서울특별시 도봉구 마들로 620,NaN,37.665430,127.046669,도봉구청,도봉경찰서,Y,1.0,25,2019-09-05,3090000,서울특별시 도봉구,0,0
1,초등학교,매헌초등학교,서울특별시 서초구 언남길 8,서울특별시 서초구 양재2동 311-1,37.471048,127.042077,서울시 서초구,서초경찰서,Y,1.0,3,2019-06-03,3210000,서울특별시 서초구,0,0


In [129]:
yellow = pd.read_csv("./옐로카펫 서울시 설치 현황(2020.2월 기준)_주소위도경도추가_최종.csv", encoding = "euc-kr")
yellow = yellow.dropna()


# 옐로카펫 설치일자 추가
try:
    for idx in range(len(child)):
        for idx2 in range(len(yellow)):
            if child["대상시설명"][idx] in yellow["설치위치(학교)"][idx2]:
                child["옐로카펫_설치일자"][idx] = yellow["설치일자"][idx2]
except Exception as e:
    print(e)

C:\Users\User\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (1,2,3,4,7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
<ipython-input-129-1476859c5e41>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  child["옐로카펫_설치일자"][idx] = yellow["설치일자"][idx2]
C:\Users\User\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [130]:
year = pd.read_csv("./서울_어린이보호구역_지정년도.csv", encoding = "euc-kr")

# 숫자만 추출
for idx in range(len(year)):
    year["지정"][idx] = re.findall("[0-9]{2}", year["지정"][idx])[0]
    
year = year.iloc[:, :4]
year.head(1)

for idx in range(len(year)):
    year.loc[idx, "명칭"] = re.findall("\w{1,}", year.loc[idx, "명칭"])[0]

<ipython-input-130-ab30f2a79f40>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  year["지정"][idx] = re.findall("[0-9]{2}", year["지정"][idx])[0]


In [131]:
for idx in range(len(child)):
    for idx2 in range(len(year)):
        if year.loc[idx2, "지역"] in child.loc[idx, "소재지도로명주소"]:
            if year.loc[idx2, "명칭"] in child.loc[idx, "대상시설명"]:
                child.loc[idx, "보호구역_지정년도"] = year.loc[idx2, "지정"]

In [132]:
child.to_csv("./어린이보호구역_최종.csv", encoding = "euc-kr")

In [139]:
child["보호구역_지정년도"]

0       08
1       08
2       96
3        0
4       95
        ..
1539    12
1540    13
1541    15
1542    15
1543     0
Name: 보호구역_지정년도, Length: 1544, dtype: object

In [135]:
child[child["보호구역_지정년도"] == 0].to_csv("./어린이보호구역_확인")

(49, 16)